Econometrics II PS 3
==
Maximilian Huber


You can find this code on my GitHub account: https://github.com/MaximilianJHuber/NYU
1.
==
a.
--

In [6]:
using Optim
using DataFrames
using ForwardDiff
using Roots
using Distributions

In [2]:
data = convert(Matrix,readtable("hsdata.csv"));
dataexp = exp(data);

In [3]:
#operates on single observations!
function g(w,z,theta) 
    return (theta[1] * w[1]^(-theta[2]) * w[2] - 1) .* collect(z[:])
end

#operates on data set!
function gn(w,z,theta)
    i = 1
    result = g(w[i,:],z[i,:],theta)
    
    for i in 2:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)
    end
    return result/size(w)[1]
end

#Wrapper creates a closure around the data set provided
function Qn_wrapper(w,z)
    return theta -> 1/2*(gn(w,z,theta)'*gn(w,z,theta))[1,1]
end

Qn_wrapper (generic function with 1 method)

In [4]:
w = dataexp[2:end,[2,4]]
z = hcat(ones(size(dataexp)[1]-1), dataexp[1:end-1,4], 1 .+ data[1:end-1,2])

Qn = Qn_wrapper(w,z)
optres = optimize(Qn, [1., 4.], BFGS(), Optim.OptimizationOptions(g_tol=1e-12, autodiff=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [1.0,4.0]
 * Minimizer: [0.9949535485769525,4.439870328581574]
 * Minimum: 1.811863e-08
 * Iterations: 10
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-12: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 53
 * Gradient Calls: 53

So $\theta_A = (0.995,4.4399)$.

b.
--
I define:
$$\hat{W}_A=\left(\frac{1}{n} \sum_{t=1}^n{g(w_t,\theta_A)g(w_t,\theta_A)'}\right)^{-1}$$
as in equation (3.5).

In [5]:
thetaA = optres.minimum

function What(w,z,theta) 
    result = zeros(Float64, size(z)[2], size(z)[2])
    for i in 1:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)*g(w[i,:],z[i,:],theta)'
    end
    return inv(result/size(z)[1])
end

WhatA = What(w,z,thetaA) 

3×3 Array{Float64,2}:
  54729.1    3945.03  -57729.5
   3945.03   2427.59   -6456.5
 -57729.5   -6456.5    63401.5

In [6]:
function Qn_wrapper(w,z,What)
    return theta -> 1/2*(gn(w,z,theta)'*What*gn(w,z,theta))[1,1]
end

QnWhatA = Qn_wrapper(w,z,WhatA)
optresB = optimize(QnWhatA, thetaA, BFGS(), Optim.OptimizationOptions(g_tol=1e-12))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.9949535485769525,4.439870328581574]
 * Minimizer: [0.97852918332263,3.386000838445892]
 * Minimum: 3.513570e-04
 * Iterations: 8
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-12: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 37
 * Gradient Calls: 37

So $\theta_B = (0.9785,3.3860)$.

c.
--

I need an estimate for the asymptotic variance $\left(G'S^{-1}G\right)^{-1}$:

$$\hat{G} = \frac{1}{n} \sum_{t=1}^n{\frac{\partial d(w_t,\theta_B)}{\partial \theta}} = \frac{\partial \frac{1}{n} \sum_{t=1}^n{d(w_t,\theta_B)}}{\partial \theta} = \frac{\partial gn(\theta_B)}{\partial \theta}$$ and 
$$\hat{S} = \hat{W}_B$$

I use automatic differentiation (rather than numeric differentiation) for all calculations (see, http://www.juliadiff.org/).

In [7]:
thetaB = optresB.minimum

#creats a closure around the data set
function gn_wrapper(w,z)
    return theta -> gn(w,z,theta)
end

dgn(theta) = ForwardDiff.jacobian(gn_wrapper(w,z),theta)

GhatB = convert(Array{Float64,2},dgn(thetaB))

3×2 Array{Float64,2}:
 1.02203  -0.0165615
 1.10959  -0.0174242
 1.03983  -0.0166105

In [8]:
Shat = inv(What(w,z,thetaB))

3×3 Array{Float64,2}:
 0.01575    0.0173241  0.0160744
 0.0173241  0.0195261  0.0177314
 0.0160744  0.0177314  0.0164239

In [9]:
aVar = inv(GhatB'*inv(Shat)*GhatB)

2×2 Array{Float64,2}:
 0.10586    5.40968
 5.40968  321.828  

In [10]:
thetaB[1] .+ sqrt(aVar[1,1]) * [-1.96, 1.96]

2-element Array{Float64,1}:
 0.340822
 1.61624 

In [11]:
thetaB[2] .+ sqrt(aVar[2,2]) * [-1.96, 1.96]

2-element Array{Float64,1}:
 -31.7755
  38.5475

d.
--
Since I do minimization of a positive function, the Hansen-Sargan statistic is:
$$J=2Qn(\theta_B)$$

In [ ]:
2*size(z)[1]*QnWhatA(thetaB)

In [12]:
1-cdf(Chisq(1),2*size(z)[1]*QnWhatA(thetaB))

0.6757269985575208

The p-value is not significant. I accept the $H_0$, that the $\mathbb{E}\left[ g(w_t;\theta_0 \right]=0$.
e.
--

In [91]:
w = dataexp[3:end,[2,4]]
z = hcat(ones(size(dataexp)[1]-2), dataexp[2:end-1,4], 1 .+ data[2:end-1,3], 1 .+ data[2:end-1,2],
        dataexp[1:end-2,4], 1 .+ data[1:end-2,3], 1 .+ data[1:end-2,2])

Qn = Qn_wrapper(w,z)
optres = optimize(Qn, [1., 4.], BFGS(), Optim.OptimizationOptions(g_tol=1e-12, autodiff=true))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [1.0,4.0]
 * Minimizer: [0.9970035183861444,4.576082645831093]
 * Minimum: 1.614946e-07
 * Iterations: 10
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-12: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 54
 * Gradient Calls: 54

In [92]:
thetaA = optres.minimum
WhatA = What(w,z,thetaA) 

7×7 Array{Float64,2}:
      2.10229e5  -49990.2        …   35729.9  -45719.0      -1.82658e5
 -49990.2             1.93473e5     -11887.1   22359.0       2.34519e5
  66704.9            -2.27365e5      17095.0  -29530.4      -2.82283e5
 -29033.0            -1.75958e5       8384.1  -16963.8      -1.91669e5
  35729.9        -11887.1            27165.7  -32152.7  -45008.2      
 -45719.0         22359.0        …  -32152.7   41287.6   60775.8      
     -1.82658e5       2.34519e5     -45008.2   60775.8       3.88417e5

In [98]:
QnWhatA = Qn_wrapper(w,z,WhatA)
optresB = optimize(QnWhatA, thetaA, BFGS(), Optim.OptimizationOptions(g_tol=1e-12))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.9970035183861444,4.576082645831093]
 * Minimizer: [0.97962853228036,3.467092989529142]
 * Minimum: 1.378672e-03
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-12: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 28
 * Gradient Calls: 28

In [99]:
thetaB = optresB.minimum
dgn(theta) = ForwardDiff.jacobian(gn_wrapper(w,z),theta)
GhatB = convert(Array{Float64,2},dgn(thetaB))
Shat = inv(What(w,z,thetaB))
aVar = inv(GhatB'*inv(Shat)*GhatB)
thetaB[1] .+ sqrt(aVar[1,1]) * [-1.96, 1.96]

2-element Array{Float64,1}:
 0.412923
 1.54633 

In [100]:
thetaB[2] .+ sqrt(aVar[2,2]) * [-1.96, 1.96]

2-element Array{Float64,1}:
 -27.9268
  34.861 

In [101]:
2*size(z)[1]*QnWhatA(thetaB)

0.683821428196953

In [102]:
1-cdf(Chisq(1),2*size(z)[1]*QnWhatA(thetaB))

0.4082739268399961

2.
==
a.
--
With 
$
\Gamma=\left(\begin{array}{cc}
1 & 0\\
-\gamma & 1
\end{array}\right)^{-1}
$, $\Sigma=\left(\begin{array}{cc}
\sigma_{c}^{2} & \rho\sigma_{c}\sigma_{r}\\
\rho\sigma_{c}\sigma_{r} & \sigma_{r}^{2}
\end{array}\right)$ and $\sigma^{2}(\gamma)=\left(\begin{array}{cc}
-\gamma & 1\end{array}\right)\Sigma\left(\begin{array}{c}
-\gamma\\
1
\end{array}\right)$ I find:

$$x_{t+1}=\left(\begin{array}{c}
c_{t+1}\\
r_{t+1}
\end{array}\right)-\Gamma\left(\begin{array}{cc}
\alpha & \beta\\
0 & 0
\end{array}\right)\left(\begin{array}{c}
c_{t}\\
r_{t}
\end{array}\right)-\Gamma\left(\begin{array}{c}
\mu_{c}\\
\lg(\delta)+\frac{1}{2}\sigma^{2}(\gamma)
\end{array}\right)\sim\mathbb{\mathcal{N}}\left(\left[\begin{array}{c}
0\\
0
\end{array}\right],\Gamma\Sigma\Gamma'\right)$$
I maximize: $$Q_{n}\left(\theta\right)=\frac{1}{n}\sum_{t=1}^{n-1}\lg\: f\left(x_{t+1}\mid x_{t};\theta\right)$$

In [19]:
w = data[1:end,[2,4]]

#theta = (δ,γ,α,β,μc,σc,σr,ρ)

#for a single observation:
@inline function m(w,theta)
    (δ,γ,α,β,μc,σc,σr,ρ) = theta
    Σ = [σc^2 ρ*σc*σr; ρ*σc*σr σr^2]
    Γ = inv([1 0; -γ 1])
    
    x = w[:] - (Γ*[α β; 0 0]*w[:]) - Γ*[μc; log(δ) + [-γ 1]*Σ*[-γ; 1]]
    return log(pdf(MvNormal([0, 0], Γ*Σ*Γ'),x))
end

function Qn_wrapper(w)
    return function(theta)
        i = 2
        result = m(w[i,:],theta)
        
        for i in 3:size(w)[1]
            result += m(w[i,:],theta)
        end
        
        return -result/size(w)[1] #I am going to minimize this function!
    
        end
end

Qn_wrapper (generic function with 1 method)

In [20]:
Qn = Qn_wrapper(w)
#theta =     (δ, γ , α,   β,  μc, σc, σr, ρ)


optres = optimize(Qn, [1,0.1,0.5,0.99,1.1,0.8,0.4,0.5], 
            NelderMead(), Optim.OptimizationOptions(iterations=5000, autodiff=true))

LoadError: LoadError: InterruptException:
while loading In[20], in expression starting on line 5

In [11]:
thetahat = optres.minimum

8-element Array{Float64,1}:
  1.04574   
  0.789876  
 -0.162603  
  0.00693633
  0.0199066 
  0.032986  
  0.132013  
  0.431393  

Since I am worried that the model is misspecified I calculate the sandwich estimator for the asymptotic variance $\hat{H}^{-1} \hat{\Sigma} \hat{H}^{-1}$ where:

$$\hat{H}=\frac{1}{n} \sum_{t=1}^{n-1} {H(w_t; \hat{\theta}}) = \frac{\partial^2 Q_n(\hat{\theta})}{\partial \theta \partial \theta'}$$

In [13]:
ForwardDiff.hessian(Qn,thetahat)

8×8 Array{Float64,2}:
   64.2119     1.04363     …    -3.29818   16.2398      -0.461386
    1.04363    0.0964241        -4.08929    1.35424      0.295367
   -2.01987   -0.00509994       -2.10164    0.0642685   -0.162918
   -8.37412   -0.0730475        -6.75836   -0.307002    -0.494004
 -115.929     -1.88422           6.23055  -29.3062       0.837223
   -3.29818   -4.08929     …  2041.08     -53.3016     -15.9303  
   16.2398     1.35424         -53.3016   131.56        -4.10459 
   -0.461386   0.295367        -15.9303    -4.10459      1.78378 